In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 447.5 kB/s eta 0:00:001m631.7 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 4.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.2 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from sentence_transformers import SentenceTransformer

# Load the model
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

# Define the user question
user_question = "I just discovered the course. Can I still join it?"

# Create the embedding
embedding = embedding_model.encode(user_question)

# Print the first value of the resulting vector
first_value = embedding[0]
print(first_value)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


0.078222655


In [3]:
!pip install -U sentence-transformers requests

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import requests

# Load the documents with ids
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Print the structure of the first document
print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}


In [4]:
from sentence_transformers import SentenceTransformer

# Load the embeddings model
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

# Assuming the key for categorization is 'course'
filtered_documents = [doc for doc in documents if doc.get('course') == 'machine-learning-zoomcamp']

# Ensure we have only 375 documents
assert len(filtered_documents) == 375

# Create embeddings for the filtered documents
embeddings = embedding_model.encode([doc['text'] for doc in filtered_documents])

# Print the first value of the resulting vector for each document embedding
for idx, embedding in enumerate(embeddings):
    print(f"Document {idx + 1} first value: {embedding[0]}")

Document 1 first value: 0.019430646672844887
Document 2 first value: 0.07348556816577911
Document 3 first value: 0.11881333589553833
Document 4 first value: -0.010515620931982994
Document 5 first value: 0.02730262093245983
Document 6 first value: 0.06365044414997101
Document 7 first value: 0.021822232753038406
Document 8 first value: 0.02610303834080696
Document 9 first value: 0.06829999387264252
Document 10 first value: 0.08126991987228394
Document 11 first value: -0.007925225421786308
Document 12 first value: 0.0035986420698463917
Document 13 first value: 0.016694674268364906
Document 14 first value: 0.09909980744123459
Document 15 first value: 0.09511077404022217
Document 16 first value: 0.014466970227658749
Document 17 first value: 0.014206071384251118
Document 18 first value: 0.027385961264371872
Document 19 first value: 0.04139387980103493
Document 20 first value: 0.013038105331361294
Document 21 first value: 0.04345778003334999
Document 22 first value: 0.08940999209880829
Docume

In [5]:
import numpy as np

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

# Create embeddings for the combined question and text fields
embeddings = []
for doc in filtered_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

print(f'The shape of X is: {X.shape}')

The shape of X is: (375, 768)


In [6]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

scores = X.dot(v)

highest_score = np.max(scores)
print(f'The highest score in the results is: {highest_score}')

The highest score in the results is: 0.6506572961807251


In [8]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

print(ground_truth[:5])

[{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Can you provide a link to sign up?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Is there an FAQ for this Machine Learning course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'Does this course have a GitHub repository for the sign-up link?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}, {'question': 'How can I structure my questions and answers for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}]


In [9]:
import requests
from sentence_transformers import SentenceTransformer
import numpy as np

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

relative_url_docs = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url_docs}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

filtered_documents = [doc for doc in documents if doc.get('course') == 'machine-learning-zoomcamp']

assert len(filtered_documents) == 375

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

embeddings = []
for doc in filtered_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)

def calculate_hitrate(search_engine, ground_truth, num_results=5):
    hits = 0
    for gt in ground_truth:
        query = gt['question']
        correct_doc_id = gt['document']  
        v_query = embedding_model.encode(query)
        results = search_engine.search(v_query, num_results=num_results)
        result_ids = [res['id'] for res in results]
        if correct_doc_id in result_ids:
            hits += 1
    hitrate = hits / len(ground_truth)
    return hitrate

hitrate = calculate_hitrate(search_engine, ground_truth, num_results=5)
print(f'Hitrate: {hitrate}')

Hitrate: 0.9398907103825137


In [15]:
from elasticsearch import Elasticsearch

es = Elasticsearch(["http://localhost:9200"])

if es.ping():
    print("Elasticsearch is up and running!")
else:
    print("Could not connect to Elasticsearch.")

Elasticsearch is up and running!


In [19]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch(["http://localhost:9200"])

index_name = 'machine-learning-zoomcamp-embeddings'

def prepare_documents(filtered_documents, embeddings):
    for i, doc in enumerate(filtered_documents):
        yield {
            "_index": index_name,
            "_id": doc['id'],
            "_source": {
                "embedding": embeddings[i],
                "question": doc['question'],
                "text": doc['text']
            }
        }

documents_to_index = list(prepare_documents(filtered_documents, embeddings))

success, failed = bulk(es, documents_to_index, refresh=True, raise_on_error=False)

if failed:
    failed_docs = [item['index']['_id'] for item in failed]
    print(f"Failed to index documents: {failed_docs}")
else:
    print("Documents indexed successfully.")


Documents indexed successfully.


In [20]:
query_vector = embedding_model.encode(user_question)
query_vector_norm = query_vector / np.linalg.norm(query_vector)

es_query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_vector_norm.tolist()}
            }
        }
    }
}

response = es.search(index=index_name, body=es_query)

if response['hits']['total']['value'] > 0:
    top_hit = response['hits']['hits'][0]
    document_id = top_hit['_id']
    score = top_hit['_score']
    print(f"Document ID with highest score: {document_id}")
    print(f"Score: {score}")
else:
    print("No documents found.")

Document ID with highest score: ee58a693
Score: 1.6506574


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_hitrate_with_elastic(embedding_model, es, index_name, ground_truth, num_results=5):
    hit_count = 0
    total_queries = len(ground_truth)

    for query in ground_truth:
        user_question = query['question']
        query_vector = embedding_model.encode(user_question)
        query_vector_norm = query_vector / np.linalg.norm(query_vector)

        # Elasticsearch search query
        es_query = {
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {"query_vector": query_vector_norm.tolist()}
                    }
                }
            },
            "size": num_results
        }

        
        response = es.search(index=index_name, body=es_query)


        if response['hits']['total']['value'] > 0:
            top_hits = [hit['_id'] for hit in response['hits']['hits']]
        else:
            top_hits = []

        # Evaluate hit rate
        if query['document'] in top_hits:
            hit_count += 1

    # Calculate hit rate
    hitrate = hit_count / total_queries
    return hitrate

hitrate_elastic = evaluate_hitrate_with_elastic(embedding_model, es, index_name, ground_truth, num_results=5)
print(f"Hitrate with Elastic: {hitrate_elastic:.4f}")

Hitrate with Elastic: 0.9399
